In [1]:
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd
import json

In [4]:
loc = 'India'

with open(f'docs/pars/pars_nods_{loc}.json', 'r') as f:
    post = json.load(f)


In [7]:
class I:
    NonTB = 0
    Asym = 1
    Sym = 2
    ExCS = 3
    TxPub = 4
    TxEng = 5
    TxPri = 6
    FpPub = 7
    FpEng = 8
    FpPri = 9

In [29]:
class Model:
    def get_y0(self, t, pars):
        k = np.exp(- pars['adr'] * (t - 2020))
        prev = k * pars['prv0']

        y0 = np.zeros(8)
        y0[I.NonTB] = 1 - prev
        y0[I.Asym] = pars['prv_a'] * k
        y0[I.Sym] = pars['prv_s'] * k
        y0[I.ExCS] = pars['prv_c'] * k

        y0.sum()
        return y0
    
    def calc(self, t, y, pars):
        calc = dict()

        calc['onset'] = onset = pars['r_sym'] *  y[I.Asym]
        calc['aware'] = pars['r_aware'] * y[I.Sym]
    
        p_det = np.array([pars['p_pub'], (1 - pars['p_pub']) * pars['p_ppm'], (1 - pars['p_pub']) * (1 - pars['p_ppm'])])
        p_txi = np.array([pars['p_txi_pub'], pars['p_txi_eng'], pars['txi_pri']])
        calc['det'] = pars['r_det'] * p_det * p_txi * y[I.ExCS]

        calc['sc_a'] = sc_a = pars['r_sc'] * y[I.Asym]
        calc['sc_s'] = pars['r_sc'] * y[I.Sym]
        calc['sc_c'] = pars['r_sc'] * y[I.ExCS]
        calc['die_a'] = die_a = pars['r_death_a'] * y[I.Asym]
        calc['die_s'] = pars['r_death_s'] * y[I.Sym]
        calc['die_c'] = pars['r_death_s'] * y[I.ExCS]

        calc['acf_a'] = acf_a = pars['r_acf'] * pars['sens_acf'] * y[I.Asym]
        calc['acf_s'] = pars['r_acf'] * pars['sens_acf'] * y[I.Sym]
        calc['acf_c'] = pars['r_acf'] * pars['sens_acf'] * y[I.ExCS]

    
        calc['inc'] = onset + acf_a + sc_a + die_a - pars['adr'] * y[I.Asym]

        return calc

    def __call__(self, t, y, pars):
        calc = self.calc(t, y, pars)
        dy = np.zeros_like(y)
        
        inc, onset, aware = calc['inc'], calc['onset'], calc['aware']
        sc_a, sc_s, sc_c = calc['sc_a'], calc['sc_s'], calc['sc_c']
        die_a, die_s, die_c = calc['die_a'], calc['die_s'], calc['die_c']
        acf_a, acf_s, acf_c = calc['acf_a'], calc['acf_s'], calc['acf_c']
        det = calc['det']
        
        dy[I.Asym] += inc - onset - acf_a - sc_a - die_a
        dy[I.Sym] += onset - aware - acf_s - sc_s - die_s
        dy[I.ExCS] += aware - det.sum() - acf_c - sc_c - die_c
        return dy
    
    def measure(self, t, y, pars):
        calc = self.calc(t, y, pars)
        mea = {'Time': t}
        mea['N'] = n = y.sum()
        mea['PrevA'] = y[I.Asym] / n
        mea['DetR'] = calc['det'].sum() / n
        return mea
    
    
    def simulate(self, pars):
        y0 = self.get_y0(2015, pars)
        ys = solve_ivp(self, [2015, 2025], y0 = y0, args = (pars, ), dense_output=True)
        ms = [self.measure(t, ys.sol(t), pars) for t in np.linspace(2015, 2025, 21)]
        ms = pd.DataFrame(ms)
        return ys, ms
        
    

In [31]:
m = Model()
ys, ms = m.simulate(post[0])

In [38]:
- np.diff(np.log(ms.PrevA)) * 2, post[0]['adr']

(array([0.0452576 , 0.04527222, 0.04527001, 0.04527583, 0.04527313,
        0.04528362, 0.04527617, 0.04529151, 0.04527819, 0.0453011 ,
        0.04528059, 0.04530771, 0.0452867 , 0.04530827, 0.04529411,
        0.04530742, 0.04530275, 0.04530991, 0.0453073 , 0.04531778]),
 0.0454)

In [77]:
ys = solve_ivp(fn, [2015, 2025], y0 = y0, args = (p, ))

In [78]:
np.diff(np.log(ys.y[I.Asym]))/ np.diff(ys.t)

array([-0.0453766, -0.0453766, -0.0453766, -0.0453766, -0.0453766,
       -0.0453766, -0.0453766, -0.0453766, -0.0453766, -0.0453766])

In [79]:
p['r_acf'] * p['sens_acf']

0.0025725450377081